# Python Code: Quality Assurance Data Task

The code here prepares the tab-delimited SAT.txt data so that it is ready for the steps in the Data Transformation process.

**Input:** SAT.txt

**Output:** SAT_clean.txt (ready for Load into MySQL for further processing e.g., join, etc.)

# Methods for Identifying Data Integrity issues

Based on the information provided and my initial inspection in Excel, these are the indicators I'm using to identify Data Integrity issues:
- Duplicate entries (these will need removal b/c each school should be a unique record for 2013).
- Null values in table (e.g., CODE values missing, etc.).
- Errors with Data Entry for Test Scores (Here I use the TOTALS column as a way to sanity check things)
- Average Test Scores are outside of the 200-800 range (range provided in layout file).

# Description of My Process and Findings during the QA Data Task:
1. As mentioned before, for time's sake I did my initial inspection in Microsoft Excel.
    - This allowed me to quickly get oriented with the data and identify ways to run my QA/cleaning of the SAT data.
2. These are the **Data Integrity issues identified**:
    - Six duplicated entries were found and dropped.
    - Four null values were found and dropped.
    - Three Data Entry Errors related to Test Scores and Total columns were identified and dropped.
    - Seven entries were found to have Test Score values outside of the range 200-800 and were dropped.   

**Note:** This brought the inititial 400 SAT records down to 380 and those were then loaded into MySQL for next steps.

## Import libraries and read in SAT data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# create path to project folder
path = r'/Users/jdexter/Desktop/dexter_data_task'

# change working directory using 'path' variable and print current directory for varification
os.chdir(path); print("Current Directory: ", os.getcwd())

Current Directory:  /Users/jdexter/Desktop/dexter_data_task


In [2]:
# Import tab-delimited text file as dataframe (df)
sat_df = pd.read_table('SAT.txt', sep='\t')

# View a sample of rows from SAT dataframe to be sure the data read in correctly
sat_df.sample(5)

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
87,7,700.0,40,1168,394,391,383,3.7,799
187,21,2580.0,40,1528,516,508,504,44.7,716
65,5,2610.0,40,1547,538,507,502,47.3,488
344,37,4960.0,50,1582,538,524,520,53.1,608
377,41,1870.0,50,1513,519,498,496,44.8,306


## Dealing with Duplicate Entries by dropping any rows that are duplicates

In [3]:
# Get count of rows and columns to see how many records we are starting with (should be 400, 9)
print('rows, columns: ', sat_df.shape)

rows, columns:  (400, 9)


In [4]:
# Count number of duplicated entries
print("The number of duplicated records: ", sat_df.duplicated().sum())

# Show the records in the dataframe where the same exact entry showed up previously in the dataset
sat_df.loc[sat_df.duplicated(), :]

The number of duplicated records:  6


,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
382,41,280.0,20,1527,509,510,508,46.6,381
383,41,1870.0,50,1513,519,498,496,44.8,306
384,41,3675.0,50,1502,514,503,485,44.3,282
385,41,4100.0,50,1543,529,513,501,44.1,782
386,41,5460.0,50,1365,460,458,447,25.0,585
387,41,5465.0,50,1517,516,505,496,46.4,61


### Let's go ahead and drop these duplicates so they don't cause issues later (e.g., with the Load or Joins)

In [5]:
# Drop duplicated entries and keep the first that appears
sat_df.drop_duplicates(keep='first', inplace=True)

# Get count of rows to double check that the 6 duplicated entries were dropped
sat_df.shape

(394, 9)

## Dealing with Null values by dropping any rows that have them

In [6]:
# A View of the SAT dataframe table info shows 4 Null values in the 
print("Number of Null values in table: ", sat_df.isnull().sum().sum())

# View the null values
sat_df[pd.isnull(sat_df).any(axis=1)]

Number of Null values in table:  4


,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
106,11,NaN,20,1199,423,386,390,5.5,361
107,11,NaN,30,1380,472,454,454,28.5,32
108,11,NaN,50,1363,464,453,446,19.4,254
109,11,NaN,50,1411,490,464,457,28.1,127


**Note:** When dealing with missing data its typically one of two things:
 
1. Data doesn't exist (more than likely not the case here).
2. Data wasn't recorded (more than likely what happened).
  - If given a list of all the codes, I would create a Data Dictionary that could pull and fill CODES as necessary to complete records and avoid Data Loss.

 
### There are only 4 rows of data that have null values and so let's drop them from the dataframe

In [7]:
# Drop na/null values 
sat_df.dropna(inplace=True)

# set float to integer data type
sat_df.DIST_CODE = sat_df.DIST_CODE.astype('int')

# Check to be sure Nulls were dropped 
print("Number of Null values left in table: ", sat_df.isnull().sum().sum())

# Get count of rows to double check that the 4 null values were dropped (count should now be 390)
sat_df.shape

Number of Null values left in table:  0


(390, 9)

## Dealing with Data Entry Errors for Test Scores by Dropping those where Totals don't match

   - Here I'm just using simple arithmetic to check and see if the sum total of the Avg. Test Score columns are equal to the value stored in the TOTAL column for each instance/record.

In [8]:
# Setup variables to simplify readability of code for TOTAL varification
math = sat_df['MATHEMATICS']
reading = sat_df['CRITICAL_READING']
writing = sat_df['WRITING']

In [9]:
# Add together the avg. scores from 3 columns and populate new column NEW_TOTAL.
sat_df['NEW_TOTAL'] = math + reading + writing

# Add column ERROR and use conditional statement to populate whether an error is present
sat_df['ERROR'] = np.where(sat_df['TOTAL'] != sat_df['NEW_TOTAL'], 'YES', 'NO')

# View rows that have Test Score Data Entry issues
sat_df.loc[sat_df['TOTAL'] != sat_df['NEW_TOTAL'], 
           ['ERROR', 'TOTAL', 'NEW_TOTAL', 'MATHEMATICS', 'CRITICAL_READING', 'WRITING']]

,ERROR,TOTAL,NEW_TOTAL,MATHEMATICS,CRITICAL_READING,WRITING
88,YES,1700,1734,594,566,574
269,YES,1410,1638,557,540,541
328,YES,1346,1402,490,452,460


### Let's go ahead and drop these 3 rows with Errors so they don't end up in the clean dataset

In [10]:
# Select only the records that had NO errors from 'totals' and store in dataframe
sat_df_errors_dropped = sat_df.loc[sat_df['ERROR'] == 'NO'].copy()

# Drop uneccesary columns
sat_df_errors_dropped.drop(['ERROR', 'NEW_TOTAL'], axis=1, inplace=True)

# Get count of rows to double check that the 4 null values were dropped (count should now be 387)
sat_df_errors_dropped.shape

(387, 9)

### Here there are a couple of possibilities  why errors exist and it would take further investigation to resolve:

 1. Someone made an error when calculating the TOTALS.
 2. Someone simply made an error when recording data in 1 or more of the 4 columns.

## Dealing with Test Score Values that are out of Range 200-800

 - A close inspection revealed there are some values less than 200 but none greater than 800
     - There were 7 of schools that had 0 scores and 0 students Scored
 - Let's go ahead and drop these records where all Test values were below 200.

In [11]:
# Make copy of dataframe 
sat_check_range = sat_df_errors_dropped.copy()

# Because we know there are a number of records with the value of zero, let's view them before dropping them 
sat_check_range.loc[sat_check_range['MATHEMATICS'] < 200]

,CO_CODE,DIST_CODE,SCH_CODE,TOTAL,MATHEMATICS,CRITICAL_READING,WRITING,SAT_1550,N_STUDENTS_SCORED
208,23,3150,300,0,0,0,0,0.0,0
209,23,3150,301,0,0,0,0,0.0,0
305,31,4010,1,0,0,0,0,0.0,0
308,31,4010,4,0,0,0,0,0.0,0
309,31,4010,5,0,0,0,0,0.0,0
362,39,4160,51,0,0,0,0,0.0,0
398,80,8010,980,0,0,0,0,0.0,0


### Let's go ahead and drop these 7 rows where Test Score values are out of range

In [12]:
# Drop rows with values below 200 and store in new dataframe
sat_clean_df = sat_check_range.loc[sat_check_range['MATHEMATICS'] > 200].copy()

# Get count of rows to double check that the 7 values that were out of range were dropped (count should now be 380)
sat_clean_df.shape

(380, 9)

## Export cleaned/QA SAT dataframe with 380 records  as tab-delimeted text file

In [13]:
# Export cleaned SAT data as tab delimeted text file
sat_clean_df.to_csv('SAT_clean.txt', sep='\t', index=False)

In [14]:
# Take a final look using dataframe info funcitonality to be sure things are looking good
sat_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 399
Data columns (total 9 columns):
CO_CODE              380 non-null int64
DIST_CODE            380 non-null int64
SCH_CODE             380 non-null int64
TOTAL                380 non-null int64
MATHEMATICS          380 non-null int64
CRITICAL_READING     380 non-null int64
WRITING              380 non-null int64
SAT_1550             380 non-null float64
N_STUDENTS_SCORED    380 non-null int64
dtypes: float64(1), int64(8)
memory usage: 29.7 KB
